In [2]:
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score
from imblearn.over_sampling import SMOTE
from sklearn.utils.class_weight import compute_class_weight

In [3]:
relevant_columns = [
    "main.disorder",  # Target variable

    # Schizophrenia (alpha PSD)
    "AB.C.alpha.a.FP1", "AB.C.alpha.b.FP2", "AB.C.alpha.c.F7", "AB.C.alpha.d.F3",
    "AB.C.alpha.e.Fz", "AB.C.alpha.f.F4", "AB.C.alpha.g.F8", "AB.C.alpha.h.T3",
    "AB.C.alpha.i.C3", "AB.C.alpha.j.Cz", "AB.C.alpha.k.C4", "AB.C.alpha.l.T4",
    "AB.C.alpha.m.T5", "AB.C.alpha.n.P3", "AB.C.alpha.o.Pz", "AB.C.alpha.p.P4",
    "AB.C.alpha.q.T6", "AB.C.alpha.r.O1", "AB.C.alpha.s.O2",

    # Trauma and stress-related disorders (beta FC)
    "COH.D.beta.a.FP1.b.FP2", "COH.D.beta.c.F7.d.F3", "COH.D.beta.e.Fz.f.F4",
    "COH.D.beta.g.F8.h.T3", "COH.D.beta.i.C3.j.Cz", "COH.D.beta.k.C4.l.T4",
    "COH.D.beta.m.T5.n.P3", "COH.D.beta.o.Pz.p.P4", "COH.D.beta.q.T6.r.O1",

    # Anxiety disorders (whole band PSD: all frequency bands)
    "AB.A.delta.a.FP1", "AB.A.delta.b.FP2", "AB.B.theta.a.FP1", "AB.B.theta.b.FP2",
    "AB.C.alpha.a.FP1", "AB.C.alpha.b.FP2", "AB.D.beta.a.FP1", "AB.D.beta.b.FP2",
    "AB.E.highbeta.a.FP1", "AB.E.highbeta.b.FP2", "AB.F.gamma.a.FP1", "AB.F.gamma.b.FP2",

    # Mood disorders (theta FC)
    "COH.B.theta.a.FP1.b.FP2", "COH.B.theta.c.F7.d.F3", "COH.B.theta.e.Fz.f.F4",
    "COH.B.theta.g.F8.h.T3", "COH.B.theta.i.C3.j.Cz", "COH.B.theta.k.C4.l.T4",
    "COH.B.theta.m.T5.n.P3", "COH.B.theta.o.Pz.p.P4", "COH.B.theta.q.T6.r.O1",

    # Addictive disorders (theta PSD)
    "AB.B.theta.a.FP1", "AB.B.theta.b.FP2", "AB.B.theta.c.F7", "AB.B.theta.d.F3",
    "AB.B.theta.e.Fz", "AB.B.theta.f.F4", "AB.B.theta.g.F8", "AB.B.theta.h.T3",
    "AB.B.theta.i.C3", "AB.B.theta.j.Cz", "AB.B.theta.k.C4", "AB.B.theta.l.T4",
    "AB.B.theta.m.T5", "AB.B.theta.n.P3", "AB.B.theta.o.Pz", "AB.B.theta.p.P4",
    "AB.B.theta.q.T6", "AB.B.theta.r.O1", "AB.B.theta.s.O2",

    # Obsessive-compulsive disorder (gamma FC)
    "COH.F.gamma.a.FP1.b.FP2", "COH.F.gamma.c.F7.d.F3", "COH.F.gamma.e.Fz.f.F4",
    "COH.F.gamma.g.F8.h.T3", "COH.F.gamma.i.C3.j.Cz", "COH.F.gamma.k.C4.l.T4",
    "COH.F.gamma.m.T5.n.P3", "COH.F.gamma.o.Pz.p.P4", "COH.F.gamma.q.T6.r.O1"
]


In [4]:
# Load dataset with only relevant columns
data = pd.read_csv("data/train.csv", usecols=relevant_columns)

# Filter only Mood Disorder and Healthy Control
data = data[data["main.disorder"].isin(["Healthy control", "Schizophrenia"])]

# Encode labels manually: Healthy Control -> 0, Mood Disorder -> 1
data["main.disorder"] = data["main.disorder"].map({
    "Healthy control": 0,
    "Schizophrenia": 1
})

# Features and labels
X = data.drop(columns=["main.disorder"]).values
y = data["main.disorder"].values.astype(int)  # Ensure integer labels

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Save the scaler
joblib.dump(scaler, "svm_schizo_scaler.pkl")

# Split into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [5]:
# ---------------- Hyperparameter Tuning (Grid Search) ----------------
param_grid = {
    "C": [0.1, 0.5, 1, 5, 10],  # Regularization parameter
    "kernel": ["linear", "rbf"],  # Linear and RBF kernel
}

svm = SVC(probability=True, class_weight="balanced", random_state=42)

grid_search = GridSearchCV(svm, param_grid, cv=5, scoring="roc_auc", n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Get the best model
best_svm = grid_search.best_estimator_

# Save the trained SVM model
joblib.dump(best_svm, "svm_schizo_tuned.pkl")

Fitting 5 folds for each of 10 candidates, totalling 50 fits


['svm_schizo_tuned.pkl']

In [6]:
# ---------------- Evaluate Best Model ----------------
y_pred = best_svm.predict(X_test)
y_pred_prob = best_svm.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_prob)

print(f"\n Tuned SVM Model for Trauma Disorder vs. Healthy Control")
print(f" Best Parameters: {grid_search.best_params_}")
print(f" Accuracy: {accuracy:.4f}")
print(f" AUC Score: {auc:.4f}")


 Tuned SVM Model for Trauma Disorder vs. Healthy Control
 Best Parameters: {'C': 0.1, 'kernel': 'linear'}
 Accuracy: 0.6500
 AUC Score: 0.7293


In [7]:
def predict_mood_disorder(new_data_path):
    """
    Loads new EEG data, scales it, and predicts Mood Disorder probability.
    """
    # Load new data
    new_data = pd.read_csv(new_data_path, usecols=relevant_columns[1:])  # Exclude "main.disorder"
    
    # Load saved scaler and model
    scaler = joblib.load("svm_schizo_scaler.pkl")
    svm = joblib.load("svm_schizo_tuned.pkl")

    # Load new data
    new_data = pd.read_csv(new_data_path, usecols=relevant_columns[1:])  # Exclude "main.disorder"

# Convert DataFrame to NumPy before applying StandardScaler
    new_data_scaled = scaler.transform(new_data.values)  # <-- FIXED


    # Predict probabilities
    predictions = svm.predict_proba(new_data_scaled)[:, 1]  # Probability of Mood Disorder

    return predictions

In [8]:
preds = predict_mood_disorder("data.csv")


In [9]:
from sklearn.metrics import roc_curve

# Compute ROC curve
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)

# Find the optimal threshold (Youden’s J statistic: max(TPR - FPR))
optimal_idx = np.argmax(tpr - fpr)
optimal_threshold = thresholds[optimal_idx]

print(f"🔹 Optimal Decision Threshold: {optimal_threshold:.4f}")

# Convert probabilities to labels using the optimal threshold
pred_labels = (y_pred_prob >= optimal_threshold).astype(int)


🔹 Optimal Decision Threshold: 0.5145


In [10]:
print(preds)

[0.95114961 0.54243625 0.63191722 0.45316052 0.41281206 0.55329271
 0.58461057 0.5749847  0.71861409 0.42004966 0.70770466 0.65877106
 0.35342861 0.41986318 0.5        0.68562113 0.66204517 0.50744768
 0.57325995 0.70229464 0.35351106 0.43668879 0.67622962 0.80373184
 0.82376438 0.56771203 0.47400552 0.57122906 0.31290611 0.51960377
 0.46819661 0.79094572 0.45962971 0.76610021 0.65954216 0.6738312
 0.48794225 0.65119998 0.57294656 0.55632919 0.43538076 0.4605318
 0.33816833 0.94749969 0.76110779 0.40646637 0.52389803 0.69677617
 0.30080091 0.57166593 0.29689743 0.8190852  0.27920948 0.68713374
 0.51123246 0.50719879 0.94724246 0.45520419 0.51741134 0.33897545
 0.55254117 0.32361236 0.39644331 0.6650027  0.68383328 0.6590403
 0.57807248 0.84732089 0.51865249 0.62764301 0.72311722 0.62055565
 0.64564217 0.90667681 0.62955276 0.6570004  0.57901716 0.71839118
 0.53222977 0.73990151 0.45813959 0.85062732 0.31780562 0.48736146
 0.34304414 0.71653816 0.49390871 0.50694165 0.51723753 0.3415360

In [12]:
def decode_predictions(preds):
    """
    Convert predictions (0,1) back to disorder labels.
    """
    label_map = {0: "Healthy control", 1: "Schizophrenia"}
    return [label_map[pred] for pred in preds]

# Example usage
pred_labels = (preds >= 0.5145).astype(int)  # Convert probabilities to binary labels
decoded_labels = decode_predictions(pred_labels)

print("\nPredicted Labels:\n", decoded_labels)


Predicted Labels:
 ['Schizophrenia', 'Schizophrenia', 'Schizophrenia', 'Healthy control', 'Healthy control', 'Schizophrenia', 'Schizophrenia', 'Schizophrenia', 'Schizophrenia', 'Healthy control', 'Schizophrenia', 'Schizophrenia', 'Healthy control', 'Healthy control', 'Healthy control', 'Schizophrenia', 'Schizophrenia', 'Healthy control', 'Schizophrenia', 'Schizophrenia', 'Healthy control', 'Healthy control', 'Schizophrenia', 'Schizophrenia', 'Schizophrenia', 'Schizophrenia', 'Healthy control', 'Schizophrenia', 'Healthy control', 'Schizophrenia', 'Healthy control', 'Schizophrenia', 'Healthy control', 'Schizophrenia', 'Schizophrenia', 'Schizophrenia', 'Healthy control', 'Schizophrenia', 'Schizophrenia', 'Schizophrenia', 'Healthy control', 'Healthy control', 'Healthy control', 'Schizophrenia', 'Schizophrenia', 'Healthy control', 'Schizophrenia', 'Schizophrenia', 'Healthy control', 'Schizophrenia', 'Healthy control', 'Schizophrenia', 'Healthy control', 'Schizophrenia', 'Healthy control', 

In [13]:
import pandas as pd

# Convert predictions into a DataFrame
output_df = pd.DataFrame({"Predicted Label": decoded_labels})

# Save to CSV
output_df.to_csv("predicted_schizo_labels.csv", index=False)

print(" Predictions saved to 'predicted_labels.csv'")


 Predictions saved to 'predicted_labels.csv'
